In [10]:
import pandas as pd
import cv2
import os
import mediapipe as mp
import time
import matplotlib.pyplot as plt

In [11]:
import pickle
model_filename = 'Model/modelsitup.pkl'
random_forest = pickle.load(open(model_filename, 'rb'))

In [12]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.6,
                    min_tracking_confidence=0.6,
                    static_image_mode=False,
                    smooth_landmarks=True,
                    model_complexity=2)

In [13]:
video_name = 'SITUPR2.MOV'
video_path = 'Test_Videos/' + video_name

In [14]:
frame_count = 0
frame_numbers = []
data_dict = {}
desired_width = 480
desired_height = 360

In [15]:
cap = cv2.VideoCapture(video_path)

In [16]:
# Assuming df is your DataFrame
def calculate_angle(row):
    import math
    row = abs(row)

    # Accessing columns by their index positions (0 to 5) from the row
    radians = math.atan2(row[5] - row[3], row[4] - row[2]) - math.atan2(row[1] - row[3], row[0] - row[2])
    angle = math.degrees(radians)
    if angle < 0:
        angle += 360
    return angle

In [17]:
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Increment frame_count
    frame_count += 1
    print(f'Processing frame {frame_count}')

    # Resize the frame to the desired width and height
    image = cv2.resize(image, (desired_width, desired_height))

    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image_rgb)

    if results.pose_landmarks:

        frame_numbers.append(frame_count)
        data_dict = {f"{landmark.name}_Point_x": [] for landmark in mp_pose.PoseLandmark}
        data_dict.update({f"{landmark.name}_Point_y": [] for landmark in mp_pose.PoseLandmark})
        data_dict.update({f"{landmark.name}_Point_z": [] for landmark in mp_pose.PoseLandmark})
        data_dict.update({f"{landmark.name}_visibility": [] for landmark in mp_pose.PoseLandmark})

        for landmark in mp_pose.PoseLandmark:
            x_list_name = f"{landmark.name}_Point_x"
            y_list_name = f"{landmark.name}_Point_y"
            z_list_name = f"{landmark.name}_Point_z"
            visibility_list_name = f"{landmark.name}_visibility"

            data_dict[x_list_name].append(results.pose_landmarks.landmark[landmark].x)
            data_dict[y_list_name].append(results.pose_landmarks.landmark[landmark].y)
            data_dict[z_list_name].append(results.pose_landmarks.landmark[landmark].z)
            data_dict[visibility_list_name].append(results.pose_landmarks.landmark[landmark].visibility)
                
        df = pd.DataFrame(data_dict)
        Angles_df = pd.DataFrame()
            
        Angle_At_Right_Elbow = df[[
            'RIGHT_WRIST_Point_x', 'RIGHT_WRIST_Point_y',
            'RIGHT_ELBOW_Point_x', 'RIGHT_ELBOW_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y'
        ]]
        
        Angles_df['Angle_At_Right_Elbow'] = Angle_At_Right_Elbow.apply(calculate_angle, axis=1)
        
        Angle_At_Left_Elbow = df[[
            'LEFT_WRIST_Point_x', 'LEFT_WRIST_Point_y',
            'LEFT_ELBOW_Point_x', 'LEFT_ELBOW_Point_y',
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y'
        ]]
        Angles_df['Angle_At_Left_Elbow'] = Angle_At_Left_Elbow.apply(calculate_angle, axis=1)
        
        # Calculate angle at right shoulder
        Angle_At_Right_Shoulder = df[[
            'RIGHT_ELBOW_Point_x', 'RIGHT_ELBOW_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y',
            'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y'
        ]]
        Angles_df['Angle_At_Right_Shoulder'] = Angle_At_Right_Shoulder.apply(calculate_angle, axis=1)
        
        # Calculate angle at left shoulder
        Angle_At_Left_Shoulder = df[[
            'LEFT_ELBOW_Point_x', 'LEFT_ELBOW_Point_y',
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
            'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y'
        ]]
        Angles_df['Angle_At_Left_Shoulder'] = Angle_At_Left_Shoulder.apply(calculate_angle, axis=1)
        
        # Calculate angle at right knee (example)
        Angle_At_Right_Knee = df[[
            'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y',
            'RIGHT_KNEE_Point_x', 'RIGHT_KNEE_Point_y',
            'RIGHT_ANKLE_Point_x', 'RIGHT_ANKLE_Point_y'
        ]]
        Angles_df['Angle_At_Right_Knee'] = Angle_At_Right_Knee.apply(calculate_angle, axis=1)
        
        # Calculate angle at left knee (example)
        Angle_At_Left_Knee = df[[
            'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y',
            'LEFT_KNEE_Point_x', 'LEFT_KNEE_Point_y',
            'LEFT_ANKLE_Point_x', 'LEFT_ANKLE_Point_y'
        ]]
        Angles_df['Angle_At_Left_Knee'] = Angle_At_Left_Knee.apply(calculate_angle, axis=1)
        
        # Calculate angle at right hip
        Angle_At_Right_Hip = df[[
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y',
            'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y',
            'RIGHT_KNEE_Point_x', 'RIGHT_KNEE_Point_y'
        ]]
        Angles_df['Angle_At_Right_Hip'] = Angle_At_Right_Hip.apply(calculate_angle, axis=1)
        
        # Calculate angle at left hip
        Angle_At_Left_Hip = df[[
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
            'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y',
            'LEFT_KNEE_Point_x', 'LEFT_KNEE_Point_y'
        ]]
        Angles_df['Angle_At_Left_Hip'] = Angle_At_Left_Hip.apply(calculate_angle, axis=1)
        
        # Calculate angle at neck (example)
        Angle_At_Neck = df[[
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
            'NOSE_Point_x', 'NOSE_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y'
        ]]
        Angles_df['Angle_At_Neck'] = Angle_At_Neck.apply(calculate_angle, axis=1)
        
        prediction = random_forest.predict(Angles_df)
        
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        
        cv2.putText(image_bgr, f"Prediction: {prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        cv2.imshow('MediaPipe Pose', image_bgr)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing frame 11
Processing frame 12
Processing frame 13
Processing frame 14
Processing frame 15
Processing frame 16
Processing frame 17
Processing frame 18
Processing frame 19
Processing frame 20
Processing frame 21
Processing frame 22
Processing frame 23
Processing frame 24
Processing frame 25
Processing frame 26
Processing frame 27
Processing frame 28
Processing frame 29
Processing frame 30
Processing frame 31
Processing frame 32
Processing frame 33
Processing frame 34
Processing frame 35
Processing frame 36
Processing frame 37
Processing frame 38
Processing frame 39
Processing frame 40
Processing frame 41
Processing frame 42
Processing frame 43
Processing frame 44
Processing frame 45
Processing frame 46
Processing frame 47
Processing frame 48
Processing frame 49
Processing frame 50
Processin

In [18]:
cap.release()
cv2.destroyAllWindows()